<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>07. Working with large datasets</h2></div>

In [ ]:
import pandas as pd
import holoviews as hv
import dask.dataframe as dd
import datashader as ds
import geoviews as gv

from holoviews.operation.datashader import datashade, aggregate
from holoviews.plotting.util import fire
datashade.cmap = fire
hv.extension('bokeh')



<p>
Dask DataFrame is provides a functionally equivalent API to pandas but allows working with data out of core and scales out to many processors and even clusters. Here we will use it to load a large CSV files of taxi coordinates.
</p>

<div >
    <img align="left" src="./assets/numba.png" width='140px'></img>
<img align="left" src="./assets/dask.svg" width='115px'></img>
<img align="left" src="./assets/datashader.png" width='158px'></img>
<img align="left" src="./assets/holoviews.png" width='140px'></img>
</div>


## Load the data

We will load our CSV data with [dask](dask.pydata.org). Dask allows working with datasets larger than fit in memory at one time but also allows parallelizing computations across multiple chunks, which can be processed on multiple threads on your local machine or scaled out to a cluster with 1000s of cores.

Dask conveniently provides a DataFrame which does a great job at replicating the pandas DataFrame API. We will load this dataset and persist it, which will load it into memroy, if your machine is low in RAM do not persist the data!

In [ ]:
ddf = dd.read_csv('../data/nyc_taxi.csv', parse_dates=['tpep_pickup_datetime'])
ddf['hour'] = ddf.tpep_pickup_datetime.dt.hour

# If your machine is low on RAM (<8GB) don't persist (everything will be much slower)
ddf = ddf.persist()
print('%s Rows' % len(ddf))
print('Columns:', list(ddf.columns))

## Create a dataset

In previous have already seen how to declare a set of [``Points``](http://holoviews.org/reference/elements/bokeh/Points.html) from a DataFrame, this works much the same, we pass in the DataFrame along with the key dimensions. Remember however we have 12 million rows of data, no plotting program will handle this well! Therefore we will use the ``datashader`` operation which will aggregate the data on a 2D grid and then apply shading, leaving us with an ``RGB`` Element to display:

In [ ]:
%opts RGB [width=800 height=400]
points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
datashade(points)

If you zoom in you will have noted that the plot rerenders depending on the zoom level. This is because the datashade operation is a dynamic operation that also declares some linked streams. These are automatically instantiated and supply the ``x_range`` and ``y_range`` to the operation, which dynamically change as you zoom in.

In [ ]:
datashade.streams

#### Exercise 1: Plot pickups

* Create a [``Points``](http://holoviews.org/reference/elements/bokeh/Points.html) Element with the 'pickup_x' and 'pickup_y' as key dimensions
* Apply the datashade operation to the points
* Optional: Change the ``cmap`` to ``inferno``

In [ ]:
from colorcet import inferno


## Adding a tile source

Using the GeoViews extension for HoloViews we can display a tile source in the background. Declare a bokeh WMTSTileSource and pass it to the gv.WMTS Element, then we can overlay it:

In [ ]:
%opts RGB [xaxis=None yaxis=None]
import geoviews as gv
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = WMTSTileSource(url=url)
gv.WMTS(wmts) * datashade(points)

#### Exercise 2: Overlay on Wikipedia tile source

* Create a bokeh ``WMTSTileSource`` with the URL provided below
* Overlay the datashaded ``points`` on top of a ``gv.WMTS`` Element

In [ ]:
url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'


## Aggregating with a variable

So far we have simply been counting taxi dropoffs, but our dataset is much richer than that. We have information about a number of variables including the total cost of a taxi ride, the ``total_amount``. Datashader provides a number of ``aggregator`` functions, which you can supply to the datashade operation. Here use the ``ds.mean`` aggregator to compute the average cost of a trip at a dropoff location:

In [ ]:
selected = points.select(total_amount=(None, 1000))
selected.data = selected.data.persist()
gv.WMTS(wmts) * datashade(selected, aggregator=ds.mean('total_amount'))

### Exercise 3: Use an aggregator

* Inspect the ``ddf`` dataframe for other variables like 'tip_amount'
* Use another aggregator such as ``ds.min`` or ``ds.max``
* Generate a datashaded plot of the points with the aggregator

## Grouping by a variable

In [ ]:
%opts Image [width=600 height=300 logz=True xaxis=None yaxis=None]
taxi_ds = hv.Dataset(ddf)
grouped = taxi_ds.to(hv.Points, ['dropoff_x', 'dropoff_y'], groupby=['hour'], dynamic=True)
aggregate(grouped).redim.values(hour=range(24))

#### Exercise 4: Facet the data

* Reuse the ``taxi_ds`` from above
* Select a subset of hours, e.g. in the morning
* Group the data by hour using the ``.to`` method
* Use a [``NdLayout``](http://holoviews.org/reference/containers/bokeh/GridSpace.html) to facet the data

In [ ]:
taxi_ds


## Additional features

We can overlay an invisible ``QuadMesh`` to reveal information on hover.

In [ ]:
%%opts QuadMesh [width=800 height=400 tools=['hover']] (alpha=0 hover_line_alpha=1 hover_fill_alpha=0)
hover_info = aggregate(points, width=40, height=20, streams=[hv.streams.RangeXY]).map(hv.QuadMesh, hv.Image)
gv.WMTS(wmts) * datashade(points) * hover_info

## Read more:

* Read the user guide on [Working with large data using datashader](http://holoviews.org/user_guide/Large_Data.html)
* See a [bokeh app](http://holoviews.org/gallery/apps/bokeh/nytaxi_hover.html) using this dataset and an additional linked stream.